In [1]:
%matplotlib inline

import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd 
import numpy as np
import gc

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import warnings
warnings.filterwarnings(action='ignore')

In [2]:
from General import *
from ReadingTheDataUtils import *
from Classifiers import *
from PreproccesUtils import *
from PerformanceEvalutionUtils import *

# Make the notebook automatically reload external python modules
%load_ext autoreload
%autoreload 2

2019-12-07 16:29:41,254 - DEBUG - MainThread - Classifiers - <module> - First time initialize logger!


In [3]:
from pathlib import Path

ROOT_PATH = Path('..')
SRC_PATH = Path('.')
DATA_PATH = ROOT_PATH / 'data'
CSV_PATH = DATA_PATH / 'ExtraSensory.per_uuid_features_labels'
CSV_SUFFIX = '.features_labels.csv'
ORIGINAL_LABLES_CSV_PATH = DATA_PATH / 'ExtraSensory.per_uuid_original_labels'
ORIGINAL_LABLES_CSV_SUFFIX = '.original_labels.csv'
FOLD_PATH = DATA_PATH / 'cv_5_folds'

# Data Exploration

In [4]:
%%time
data = pd.read_csv(DATA_PATH / 'dataset.csv', index_col='uuid')

Wall time: 22 s


In [5]:
data.head()

,timestamp,raw_acc:magnitude_stats:mean,raw_acc:magnitude_stats:std,raw_acc:magnitude_stats:moment3,raw_acc:magnitude_stats:moment4,raw_acc:magnitude_stats:percentile25,raw_acc:magnitude_stats:percentile50,raw_acc:magnitude_stats:percentile75,raw_acc:magnitude_stats:value_entropy,raw_acc:magnitude_stats:time_entropy,...,discrete:time_of_day:between0and6,discrete:time_of_day:between3and9,discrete:time_of_day:between6and12,discrete:time_of_day:between9and15,discrete:time_of_day:between12and18,discrete:time_of_day:between15and21,discrete:time_of_day:between18and24,discrete:time_of_day:between21and3,label,label_name
uuid,,,,,,,,,,,,,,,,,,,,,
00EABED2-271D-49D8-B599-1D4A09240601,1444079161,0.996815,0.003529,-0.002786,0.006496,0.995203,0.996825,0.998502,1.748756,6.684605,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1,original_label:SITTING
00EABED2-271D-49D8-B599-1D4A09240601,1444079221,0.996864,0.004172,-0.003110,0.007050,0.994957,0.996981,0.998766,1.935573,6.684603,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1,original_label:SITTING
00EABED2-271D-49D8-B599-1D4A09240601,1444079281,0.996825,0.003667,0.003094,0.006076,0.994797,0.996614,0.998704,2.031780,6.684605,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1,original_label:SITTING
00EABED2-271D-49D8-B599-1D4A09240601,1444079341,0.996874,0.003541,0.000626,0.006059,0.995050,0.996907,0.998690,1.865318,6.684605,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1,original_label:SITTING
00EABED2-271D-49D8-B599-1D4A09240601,1444079431,0.997371,0.037653,0.043389,0.102332,0.995548,0.996860,0.998205,0.460806,6.683904,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1,original_label:SITTING


In [7]:
data.label_name.unique()

array(['original_label:SITTING', 'original_label:WALKING',
       'original_label:STANDING_AND_MOVING', 'original_label:LYING_DOWN',
       'original_label:STANDING_IN_PLACE', 'original_label:RUNNING',
       'original_label:BICYCLING'], dtype=object)

In [10]:
data.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
Index: 377346 entries, 00EABED2-271D-49D8-B599-1D4A09240601 to FDAA70A1-42A3-4E3F-9AE3-3FDA412E03BF
Data columns (total 228 columns):
timestamp                                                                  377346 non-null int64
raw_acc:magnitude_stats:mean                                               377056 non-null float64
raw_acc:magnitude_stats:std                                                377056 non-null float64
raw_acc:magnitude_stats:moment3                                            377056 non-null float64
raw_acc:magnitude_stats:moment4                                            377056 non-null float64
raw_acc:magnitude_stats:percentile25                                       377056 non-null float64
raw_acc:magnitude_stats:percentile50                                       377056 non-null float64
raw_acc:magnitude_stats:percentile75                                       377056 non-null float64
raw_acc:magnitude_stats:value_entropy 

In [11]:
data['label'] = data['label'].astype('category')

for col in data.columns:
    if col.startswith('discrete'):
        data[col] = data[col].astype('category')

In [12]:
data.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
Index: 377346 entries, 00EABED2-271D-49D8-B599-1D4A09240601 to FDAA70A1-42A3-4E3F-9AE3-3FDA412E03BF
Data columns (total 228 columns):
timestamp                                                                  377346 non-null int64
raw_acc:magnitude_stats:mean                                               377056 non-null float64
raw_acc:magnitude_stats:std                                                377056 non-null float64
raw_acc:magnitude_stats:moment3                                            377056 non-null float64
raw_acc:magnitude_stats:moment4                                            377056 non-null float64
raw_acc:magnitude_stats:percentile25                                       377056 non-null float64
raw_acc:magnitude_stats:percentile50                                       377056 non-null float64
raw_acc:magnitude_stats:percentile75                                       377056 non-null float64
raw_acc:magnitude_stats:value_entropy 

# Model 

### Learn all classifiers by folds 

In [8]:
train_folds_list, test_folds_list = get_folds_list(FOLD_PATH)

In [9]:
train_folds_list, test_folds_list

([['33A85C34-CFE4-4732-9E73-0A7AC861B27A',
   '40E170A7-607B-4578-AF04-F021C3B0384A',
   '481F4DD2-7689-43B9-A2AA-C8772227162B',
   '4E98F91F-4654-42EF-B908-A3389443F2E7',
   '59818CD2-24D7-4D32-B133-24C2FE3801E5',
   '59EEFAE0-DEB0-4FFF-9250-54D2A03D0CF2',
   '61359772-D8D8-480D-B623-7C636EAD0C81',
   '665514DE-49DC-421F-8DCB-145D0B2609AD',
   '74B86067-5D4B-43CF-82CF-341B76BEA0F4',
   '797D145F-3858-4A7F-A7C2-A4EB721E133C',
   '806289BC-AD52-4CC1-806C-0CDB14D65EB6',
   '81536B0A-8DBF-4D8A-AC24-9543E2E4C8E0',
   '86A4F379-B305-473D-9D83-FC7D800180EF',
   '96A358A0-FFF2-4239-B93E-C7425B901B47',
   '99B204C0-DD5C-4BB7-83E8-A37281B8D769',
   'A5A30F76-581E-4757-97A2-957553A2C6AA',
   'A7599A50-24AE-46A6-8EA6-2576F1011D81',
   'B7F9D634-263E-4A97-87F9-6FFB4DDCB36C',
   'BEF6C611-50DA-4971-A040-87FB979F3FC1',
   'CCAF77F0-FABB-4F2F-9E24-D56AD0C5A82F',
   'F50235E0-DD67-4F2A-B00B-1F31ADA998B9',
   '2C32C23E-E30C-498A-8DD2-0EFB9150A02E',
   '3600D531-0C55-44A7-AE95-A7A38519464E',
   '4FC3214

In [13]:
def get_cols_to_scale(i_X_fold_train):
    cols_to_scale = i_X_fold_train.select_dtypes(exclude=['category']).columns
    categorial_cols = i_X_fold_train.select_dtypes(include=['category']).columns
    
    return cols_to_scale, categorial_cols

def get_X_data_dummies(i_X_fold_train, i_X_fold_test):
    X_train = pd.get_dummies(i_X_fold_train, dummy_na=False).reset_index(drop=True, inplace=False)
    X_test = pd.get_dummies(i_X_fold_test, dummy_na=False).reset_index(drop=True, inplace=False)
    
    return X_train, X_test

def data_preproccessing(i_data, i_train_folds_list, i_test_folds_list):
    # Get folds as DataFrame
    train_fold_df, test_fold_df = get_folds_train_and_test(i_data, i_train_folds_list, i_test_folds_list)
    X_fold_train, X_fold_test, y_fold_train, y_fold_test = \
            split_fold_data_to_features_and_labels(train_fold_df, test_fold_df)
    
    # Decide the scaling method, Numeric standard, categorial one-hot
    cols_to_scale, categorial_cols = get_cols_to_scale(X_fold_train)
    standard_X_train, standard_X_test = standard_data_scaling(X_fold_train[cols_to_scale], X_fold_test[cols_to_scale])
    dummies_X_train, dummies_X_test = get_X_data_dummies(X_fold_train[categorial_cols], X_fold_test[categorial_cols])
    
    # Concat the numeric with the categorial
    X_train = pd.concat([standard_X_train, dummies_X_train], axis=1)
    X_test = pd.concat([standard_X_test, dummies_X_test], axis=1)
    
    return X_train, X_test, y_fold_train, y_fold_test

In [12]:
# %%time
# classifiers = dict()

# for train_fold_lst, test_fold_lst in zip(train_folds_list, test_folds_list):
#     gc.collect()
    
#     # Preprocess the data
#     standard_X_train, standard_X_test, y_fold_train, y_fold_test = \
#                 data_preproccessing(data, train_fold_lst, test_fold_lst)
    
# #     handle_nulls_in_X(standard_X_train, standard_X_test)
    
#     # Learn all models sync
# #     models_tuple = learn_all_models_sync(standard_X_train, y_fold_train)

#     # Learn all model async
#     models_tuple = learn_all_models_async(standard_X_train, y_fold_train, i_c_score_grid_search=True)
    
#     # Put each fold result in the classifiers dict
#     classifiers.setdefault('single_sensor_classifier', []).append(models_tuple[0])
#     classifiers.setdefault('early_fusion_classifier', []).append(models_tuple[1])

In [13]:
# classifiers

In [14]:
import pickle

filename = 'classifiers_dict'

In [15]:
# # Write the model to a file
# with open(filename, 'wb') as outfile:
#     pickle.dump(classifiers, outfile)

# Performance evaluation

In [16]:
# Read the model from a file
with open(filename, 'rb') as infile:
    classifiers_dict = pickle.load(infile)

In [29]:
classifiers_dict

{'single_sensor_classifier': [{'Acc': LogisticRegression(C=0.1, class_weight='balanced', dual=False,
                      fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                      max_iter=1000, multi_class='warn', n_jobs=-1, penalty='l2',
                      random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                      warm_start=False),
   'Gyro': LogisticRegression(C=0.001, class_weight='balanced', dual=False,
                      fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                      max_iter=1000, multi_class='warn', n_jobs=-1, penalty='l2',
                      random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                      warm_start=False),
   'WAcc': LogisticRegression(C=0.001, class_weight='balanced', dual=False,
                      fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                      max_iter=1000, multi_class='warn', n_jobs=-1, penalty='l2',
                      ran

In [30]:
from Classifiers import NUM_OF_LABELS

def get_class_weights_in_data(i_y):
    y = np.array(i_y)
    class_counts = np.unique(y, return_counts=True)[1]
    
    if len(class_counts) != NUM_OF_LABELS:
        raise Exception(f"class_counts length is diffrent from {NUM_OF_LABELS}")
        
    return class_counts

def get_test_weights(i_y_test):
    return get_class_weights_in_data(i_y_test)
    

test_class_weights = np.zeros((NUM_OF_LABELS, ), dtype='int')


for i, (train_fold_lst, test_fold_lst) in enumerate(zip(train_folds_list, test_folds_list)):
    gc.collect()
    
    is_first_iteration = i == 0

    
    # Preprocess the data
    standard_X_train, standard_X_test, y_fold_train, y_fold_test = \
                data_preproccessing(data, train_fold_lst, test_fold_lst)
    
    test_class_weights = test_class_weights + get_test_weights(y_fold_test)
    print(test_class_weights)
    
    if is_first_iteration:
        single_sensors_states_dict, EF_states, LFA_states, LFL_states = get_states_arrays(data)
    
    # Get rows with all sensors data
    feature_names = get_feature_names(standard_X_train, ['label'])  # In this case we use the data with our label!
    sensor_names = get_sensor_names(feature_names)
    
    y_fold_test.reset_index(drop=True, inplace=True)

    for _, sensor_cols_name_in_data in sensor_names.items():
        mask = standard_X_test[sensor_cols_name_in_data].isnull().all(1)
        idx_to_drop = standard_X_test[mask].index
        
        standard_X_test.drop(idx_to_drop, axis=0, inplace=True)
        y_fold_test.drop(idx_to_drop, axis=0, inplace=True)

    handle_nulls_in_X(standard_X_train, standard_X_test)
    
    single_sensor_models = classifiers_dict['single_sensor_classifier'][i]
    
    # Eeach single sensor model
    get_single_sensor_state(single_sensors_states_dict, standard_X_test, y_fold_test, single_sensor_models)

    # LFA
    LFA_states = LFA_states + get_LFA_state(standard_X_test, y_fold_test,  single_sensor_models)
        
    # LFL
    LFL_states = LFL_states + get_LFL_state(standard_X_train, y_fold_train, standard_X_test, y_fold_test, single_sensor_models)


test_class_weights = (test_class_weights / test_class_weights.sum())

17143

[27244 25473  2068  4111  5693   123   347]


209

[61690 52423  3458  9863 10285   206  1416]


294

[102818  88975   4873  18437  15281    723   3643]


294

[150819 114441   6021  26645  19743   1266   4645]


294

[174336 136356   8028  29754  22517   1335   5020]


In [19]:
LFL_states

array([[ 51669,  37937,    983,   3242,   6090,     87,   2487],
       [111585, 107173, 181802, 175388, 181462, 191859, 194621],
       [ 10471,  18188,  20687,  15914,  14790,  15599,  10135],
       [ 34554,  44981,   4807,  13735,   5937,    734,   1036]],
      dtype=int64)

In [20]:
test_class_weights

array([0.46200569, 0.36135536, 0.0212749 , 0.07885071, 0.05967203,
       0.00353787, 0.01330344])

### Micro averaging

In [21]:
evalutions_dict = dict()

# Single sensor
feature_names = get_feature_names(standard_X_test, ['label'])  # In this case we using the data with our label!
sensor_names = get_sensor_names(feature_names)

for sensor_name in sensor_names:
    single_sesnor_state = single_sensors_states_dict[sensor_name]
    TP, TN, FP, FN = single_sesnor_state[0].sum(), single_sesnor_state[1].sum(), single_sesnor_state[2].sum(), single_sesnor_state[3].sum()
    sensitivity, specifisity, accuracy, precision, BA, F1 =\
        get_evaluations_metric_scores(TP, TN, FP, FN)
    insert_values_to_evaluations_dict(evalutions_dict, sensor_name, 
                                 sensitivity, specifisity, accuracy, precision, BA, F1)

# LFA
TP, TN, FP, FN = LFA_states[0].sum(), LFA_states[1].sum(), LFA_states[2].sum(), LFA_states[3].sum()
sensitivity, specifisity, accuracy, precision, BA, F1 =\
        get_evaluations_metric_scores(TP, TN, FP, FN)                              
insert_values_to_evaluations_dict(evalutions_dict, "LFA",
                                  sensitivity, specifisity, accuracy, precision, BA, F1)

# LFL
TP, TN, FP, FN = LFL_states[0].sum(), LFL_states[1].sum(), LFL_states[2].sum(), LFL_states[3].sum()
sensitivity, specifisity, accuracy, precision, BA, F1 =\
        get_evaluations_metric_scores(TP, TN, FP, FN) 
insert_values_to_evaluations_dict(evalutions_dict, "LFL", 
                                 sensitivity, specifisity, accuracy, precision, BA, F1)

pd.DataFrame.from_dict(evalutions_dict)

,classifier,accuracy,sensitivity,specifisity,BA,precision,F1
0,Acc,0.847446,0.466062,0.911010,0.688536,0.466062,0.466062
1,Gyro,0.829674,0.403857,0.900643,0.652250,0.403857,0.403857
2,WAcc,0.863879,0.523577,0.920596,0.722086,0.523577,0.523577
3,Loc,0.797876,0.292564,0.882094,0.587329,0.292564,0.292564
4,Aud,0.816116,0.356407,0.892734,0.624570,0.356407,0.356407
5,PS,0.840009,0.440030,0.906672,0.673351,0.440030,0.440030
6,LFA,0.877882,0.572588,0.928765,0.750676,0.572588,0.572588
7,LFL,0.854887,0.492104,0.915351,0.703728,0.492104,0.492104


### Macro averaging

In [22]:
evalutions_dict = dict()

# Single sensor
feature_names = get_feature_names(standard_X_test, ['label'])  # In this case we using the data with our label!
sensor_names = get_sensor_names(feature_names)

for sensor_name in sensor_names:
    single_sesnor_state = single_sensors_states_dict[sensor_name]
    scores_arr = np.zeros((6, ), dtype='int')

    for c in range(NUM_OF_LABELS):
        class_state = single_sesnor_state[:, c]
        TP, TN, FP, FN = class_state[0], class_state[1], class_state[2], class_state[3]
        scores_arr = scores_arr + get_evaluations_metric_scores(TP, TN, FP, FN)

    scores_arr = scores_arr / NUM_OF_LABELS
    
    sensitivity, specifisity, accuracy, precision, BA, F1 =\
            scores_arr[0], scores_arr[1], scores_arr[2], scores_arr[3], scores_arr[4], scores_arr[5]                       
    insert_values_to_evaluations_dict(evalutions_dict, sensor_name, 
                                 sensitivity, specifisity, accuracy, precision, BA, F1)

# LFA
scores_arr = np.zeros((6, ), dtype='int')

for c in range(NUM_OF_LABELS):
    class_state = LFA_states[:, c]
    TP, TN, FP, FN = class_state[0], class_state[1], class_state[2], class_state[3]
    scores_arr = scores_arr + get_evaluations_metric_scores(TP, TN, FP, FN)

scores_arr = scores_arr / NUM_OF_LABELS

sensitivity, specifisity, accuracy, precision, BA, F1 =\
    scores_arr[0], scores_arr[1], scores_arr[2], scores_arr[3], scores_arr[4], scores_arr[5]                       
insert_values_to_evaluations_dict(evalutions_dict, "LFA",
                                  sensitivity, specifisity, accuracy, precision, BA, F1)

# LFL
scores_arr = np.zeros((6, ), dtype='int')

for c in range(NUM_OF_LABELS):
    class_state = LFL_states[:, c]
    TP, TN, FP, FN = class_state[0], class_state[1], class_state[2], class_state[3]
    scores_arr = scores_arr + get_evaluations_metric_scores(TP, TN, FP, FN)

scores_arr = scores_arr / NUM_OF_LABELS

sensitivity, specifisity, accuracy, precision, BA, F1 =\
        scores_arr[0], scores_arr[1], scores_arr[2], scores_arr[3], scores_arr[4], scores_arr[5]                       
insert_values_to_evaluations_dict(evalutions_dict, "LFL", 
                                 sensitivity, specifisity, accuracy, precision, BA, F1)

pd.DataFrame.from_dict(evalutions_dict)

,classifier,accuracy,sensitivity,specifisity,BA,precision,F1
0,Acc,0.847446,0.320997,0.890292,0.605645,0.272840,0.275217
1,Gyro,0.829674,0.283291,0.881464,0.582377,0.238510,0.227424
2,WAcc,0.863879,0.418243,0.905573,0.661908,0.328287,0.347247
3,Loc,0.797876,0.219571,0.863897,0.541734,0.221638,0.177868
4,Aud,0.816116,0.236106,0.885910,0.561008,0.222260,0.194706
5,PS,0.840009,0.299159,0.901265,0.600212,0.258961,0.240554
6,LFA,0.877882,0.384194,0.910925,0.647560,0.375983,0.375271
7,LFL,0.854887,0.390825,0.911961,0.651393,0.316612,0.311716


### Weighted averaging

In [23]:
evalutions_dict = dict()

# Single sensor
feature_names = get_feature_names(standard_X_test, ['label'])  # In this case we using the data with our label!
sensor_names = get_sensor_names(feature_names)

for sensor_name in sensor_names:
    single_sesnor_state = single_sensors_states_dict[sensor_name]
    scores_arr = []

    for c in range(NUM_OF_LABELS):
        class_state = single_sesnor_state[:, c]
        TP, TN, FP, FN = class_state[0], class_state[1], class_state[2], class_state[3]
        
        scores_arr.append(get_evaluations_metric_scores(TP, TN, FP, FN))

    scores_arr = np.dot(test_class_weights, np.array(scores_arr))

    sensitivity, specifisity, accuracy, precision, BA, F1 =\
            scores_arr[0], scores_arr[1], scores_arr[2], scores_arr[3], scores_arr[4], scores_arr[5]                       
    insert_values_to_evaluations_dict(evalutions_dict, sensor_name, 
                                 sensitivity, specifisity, accuracy, precision, BA, F1)

# LFA
scores_arr = []

for c in range(NUM_OF_LABELS):
    class_state = LFA_states[:, c]
    TP, TN, FP, FN = class_state[0], class_state[1], class_state[2], class_state[3]
    
    scores_arr.append(get_evaluations_metric_scores(TP, TN, FP, FN))

scores_arr = np.dot(test_class_weights, np.array(scores_arr))
    
sensitivity, specifisity, accuracy, precision, BA, F1 =\
        scores_arr[0], scores_arr[1], scores_arr[2], scores_arr[3], scores_arr[4], scores_arr[5]                       
insert_values_to_evaluations_dict(evalutions_dict, "LFA",
                                  sensitivity, specifisity, accuracy, precision, BA, F1)

# LFL
scores_arr = []

for c in range(NUM_OF_LABELS):
    class_state = LFL_states[:, c]
    TP, TN, FP, FN = class_state[0], class_state[1], class_state[2], class_state[3]
    
    scores_arr.append(get_evaluations_metric_scores(TP, TN, FP, FN))

scores_arr = np.dot(test_class_weights, np.array(scores_arr))

sensitivity, specifisity, accuracy, precision, BA, F1 =\
        scores_arr[0], scores_arr[1], scores_arr[2], scores_arr[3], scores_arr[4], scores_arr[5]                       
insert_values_to_evaluations_dict(evalutions_dict, "LFL", 
                                 sensitivity, specifisity, accuracy, precision, BA, F1)

pd.DataFrame.from_dict(evalutions_dict)

,classifier,accuracy,sensitivity,specifisity,BA,precision,F1
0,Acc,0.690054,0.483098,0.754388,0.618743,0.503266,0.478279
1,Gyro,0.666715,0.420089,0.755224,0.587656,0.469052,0.426269
2,WAcc,0.737643,0.532560,0.812467,0.672514,0.576061,0.550077
3,Loc,0.616052,0.318200,0.730362,0.524281,0.402982,0.288002
4,Aud,0.698804,0.375249,0.838473,0.606861,0.516329,0.412965
5,PS,0.747443,0.450665,0.871141,0.660903,0.601995,0.510713
6,LFA,0.753133,0.584696,0.798422,0.691559,0.592048,0.585443
7,LFL,0.769772,0.500836,0.893784,0.697310,0.662764,0.560879


# =====================================

In [24]:
def train_model_pipeline(i_X_train, i_y_train, i_pipeline, i_param_grid=None):
    if param_grid:
        clf = GridSearchCV(i_pipeline, param_grid=i_param_grid, n_jobs=1, verbose=1, scoring='f1_weighted')
        
    print(f'i_X_train.shape = {i_X_train.shape}')
    print(f'i_y_train.shape = {i_y_train.shape}')

    clf.fit(i_X_train, i_y_train)
    
    return clf

# EF training using the pipelines!

In [25]:
ef_models = []
ef_full_pipeline = [] 

In [31]:
from sklearn.compose import make_column_transformer, ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

i = 0
for train_fold_lst, test_fold_lst in zip(train_folds_list, test_folds_list):
    gc.collect()
    if i != 4:
        i += 1
        continue
    train_fold_df, test_fold_df = get_folds_train_and_test(data, train_fold_lst, test_fold_lst)
    
    X_fold_train, X_fold_test, y_fold_train, y_fold_test = \
            split_fold_data_to_features_and_labels(train_fold_df, test_fold_df)
    
    cat_cols = X_fold_train.select_dtypes(include=["category"]).columns
    cat_colds_indices = [X_fold_train.columns.get_loc(c) for c in cat_cols if c in X_fold_train]
    num_cols = X_fold_train.select_dtypes(exclude=["category"]).columns
    num_cols_indices = [X_fold_train.columns.get_loc(c) for c in num_cols if c in X_fold_train]
    
    column_transformer_sacler = ColumnTransformer([('ohe', OneHotEncoder(sparse=False), cat_colds_indices),
                       ('scaler', StandardScaler(), num_cols_indices)])
    column_transformer_null_handler = ColumnTransformer([('SimpleImputerCat', SimpleImputer(strategy="most_frequent"), cat_colds_indices),
                       ('SimpleImputerNum', SimpleImputer(strategy='mean'), num_cols_indices)])
    pipe_preprocess = Pipeline(
        [("column_transformer_sacler", column_transformer_sacler),
                                ("column_transformer_null_handler",
                                 column_transformer_null_handler)])
    
    pipe_learner = Pipeline(
        [("pipe_preprocess", pipe_preprocess),
         ("model", LogisticRegression())], memory="cached")
    
    C_options = [0.001, 0.01]
#     C_options = [0.001, 0.01, 0.1, 1, 10, 100]
    param_grid = [{'model__C': C_options,
                    'model__solver': ['lbfgs'],
                    'model__max_iter': [1],
                    'model__class_weight': ["balanced"]}]
    
    trained_pipe = train_model_pipeline(X_fold_train, y_fold_train, pipe_learner, param_grid)
    trained_model = trained_pipe.best_estimator_['model']
    
    ef_models.append(trained_model)
    ef_full_pipeline.append(trained_pipe)


14739

0

0

0

0

i_X_train.shape = (323580, 225)
i_y_train.shape = (323580,)
Fitting 3 folds for each of 2 candidates, totalling 6 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


ValueError: unknown categorical feature present [1 1] during transform.

# ========================================

In [34]:
cat_cols = X_fold_train.select_dtypes(include=["category"]).columns
cat_colds_indices = [X_fold_train.columns.get_loc(c) for c in cat_cols if c in X_fold_train]
num_cols = X_fold_train.select_dtypes(exclude=["category"]).columns
num_cols_indices = [X_fold_train.columns.get_loc(c) for c in num_cols if c in X_fold_train]
    
column_transformer_sacler = ColumnTransformer([('ohe', OneHotEncoder(sparse=False), cat_colds_indices),
                   ('scaler', StandardScaler(), num_cols_indices)])
column_transformer_null_handler = ColumnTransformer([('SimpleImputerCat', SimpleImputer(strategy="most_frequent"), cat_colds_indices),
                   ('SimpleImputerNum', SimpleImputer(strategy='mean'), num_cols_indices)])
pipe_preprocess = Pipeline(
    [("column_transformer_sacler", column_transformer_sacler),
                            ("column_transformer_null_handler",
                             column_transformer_null_handler)])

pipe_learner = Pipeline(
    [("pipe_preprocess", pipe_preprocess),
     ("model", LogisticRegression())], memory="cached")

C_options = [0.001, 0.01]
#C_options = [0.001, 0.01, 0.1, 1, 10, 100]
param_grid = [{'model__C': C_options,
                'model__solver': ['lbfgs'],
                'model__max_iter': [1],
                'model__class_weight': ["balanced"]}]

# trained_pipe = train_model_pipeline(X_fold_train, y_fold_train, pipe_learner, param_grid)

grid = GridSearchCV(LogisticRegression(), param_grid=param_grid, n_jobs=1, verbose=1, scoring='f1_weighted')

pipe_learner.set_params(model=grid)

Pipeline(memory='cached',
         steps=[('pipe_preprocess',
                 Pipeline(memory=None,
                          steps=[('column_transformer_sacler',
                                  ColumnTransformer(n_jobs=None,
                                                    remainder='drop',
                                                    sparse_threshold=0.3,
                                                    transformer_weights=None,
                                                    transformers=[('ohe',
                                                                   OneHotEncoder(categorical_features=None,
                                                                                 categories=None,
                                                                                 drop=None,
                                                                                 dtype=<class 'numpy.float64'>,
                                                                 